In [1]:
import os
import sys
os.chdir('..')
sys.path.append(os.getcwd())

In [ ]:
import torch
from Models.pretrain import *
from Models.cPB import *
from river import stream
from evaluation.cl_evaluation import EvaluateContinualLearning
from evaluation.learner_config import LearnerConfig
from evaluation.prequential_evaluation import EvaluatePrequential, make_dir
import traceback
from evaluation.test_utils import *

import torch.nn.functional as F
import numpy as np
import pandas as pd
import pickle
import argparse
from torch.autograd import Variable
import pprint
import copy
import warnings

warnings.filterwarnings("ignore", category=FutureWarning, module='torch.storage')
warnings.filterwarnings("ignore", category=UserWarning, module='torch.storage')
warnings.filterwarnings("ignore", category=DeprecationWarning, module='torch.storage')




In [ ]:
hidden_size_list = [50]
pretrain_mask_list = ['2']
for hidden in hidden_size_list:
  for pretrain in pretrain_mask_list:
    for iter in range(1,11):
      ###### Config the parameters ######
      base_model = 'GRU'
      batch_size = 1
      hidden_size = hidden
      seq_len = 10
      epoch_size=10
      lr = 0.01
      number_of_tasks=4
      mask_selection_NofBatch = 50 * 128
      input_size=2
      many_to_one = True
      iteration=iter
      mask_init='uniform'
      dataset='SINE'
      dataset_name = "sine_rw10_mode5_extended_16-16_1234"
      Pretrain_task = pretrain
      ###### End of Config the parameters ######
      df = pd.read_csv(os.path.join(f"datasets/{dataset}/", f"{dataset_name}.csv"))
      pretrain_model_addr=f'Performance/Pretrain/{base_model}/{dataset}/After/{dataset}-Task_{Pretrain_task}-{base_model}-pretrain-hidden{hidden_size}-epoch10_iter{iteration}.pickle'
      mask_weights=[] #if we have initial masks then reload it here


      mask_option = 'SUM'
      model_type = 'cPB'
      batch_first = True
      low_rank = False
      first_batch = True
      model=cPB(lr = lr ,hidden_size = hidden_size, seq_len = seq_len,base_model = base_model, pretrain_model_addr = pretrain_model_addr,
                mask_weights = mask_weights, mask_init = mask_init, number_of_tasks = number_of_tasks, epoch_size = epoch_size,
                input_size = input_size, mask_option = mask_option, model_type = model_type, batch_first = batch_first, low_rank = low_rank, many_to_one = many_to_one)


      # loop for each task
      x_batch = []
      y_batch = []
      prev_data_x = []
      prev_data_y = []
      for task in range(1, df["task"].max() + 1):
        df_task = df[df["task"] == task]
        df_task = df_task.drop(columns="task")
        # loop based on each batch of data
        batch_cont=0
        x = []
        y = []
        model.weights_copy_anytime(task)
        for i in range(0, len(df_task), batch_size):
          # x = df_task.iloc[0 : i + batch_size, 0:-1].values.astype(np.float32)
          # y = list(df_task.iloc[0 : i + batch_size, -1])
          if len(x_batch)==128:
            if first_batch==True:
              first_batch = False
            x_batch = []
            y_batch = []
            if first_batch == False:
              prev_data_x = x[-seq_len:]
              prev_data_y = y[-seq_len:]
          x_batch.append(df_task.iloc[i, 0:-1].values.astype(np.float32))
          y_batch.append(df_task.iloc[i, -1])
          
          x.append(df_task.iloc[i, 0:-1].values.astype(np.float32))
          y.append(df_task.iloc[i, -1])
          
          # print(len(x))
          if len(x) >= seq_len:
            if i<mask_selection_NofBatch:
              for mask_index in range (0, task):
                model.predict_one(x[-seq_len:],y[-seq_len:],mask_index,task,mask_selection=True)
                if len(x_batch)==128:
                  if first_batch:
                    model.learn_many_anytime(x_batch,y_batch,mask_index)
                  else:
                    new_batch_x = np.concatenate([prev_data_x, x_batch], axis=0) 
                    new_batch_y = np.concatenate([prev_data_y, y_batch], axis=0)           
                    model.learn_many_anytime(new_batch_x,new_batch_y,mask_index)

                  
              batch_cont+=1
            elif i==mask_selection_NofBatch:
              best_mask_index=model.add_new_column_anytime(task)
              model.predict_one(x[-seq_len:],y[-seq_len:],best_mask_index,task,mask_selection=False)
              if len(x_batch)==128:
                  new_batch_x = np.concatenate([prev_data_x, x_batch], axis=0) 
                  new_batch_y = np.concatenate([prev_data_y, y_batch], axis=0)
                  model.learn_many_anytime(new_batch_x,new_batch_y,best_mask_index)
              batch_cont+=1
            elif i>mask_selection_NofBatch:
              batch_cont+=1
              model.predict_one(x[-seq_len:],y[-seq_len:],best_mask_index,task,mask_selection=False)
              if len(x_batch)==128:
                  new_batch_x = np.concatenate([prev_data_x, x_batch], axis=0) 
                  new_batch_y = np.concatenate([prev_data_y, y_batch], axis=0)
                  model.learn_many_anytime(new_batch_x,new_batch_y,best_mask_index)
        all_y = df_task.iloc[:, -1].reset_index(drop=True)
        model.save_final_metrics_anytime(task,all_y)
        model.final_weights_saving()


      file_path = f'Performance/Results/{model_type}/{dataset}/Anytime/pretrain-T{Pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-Mask_{mask_option}_iter{iteration}.pkl'
      with open(file_path, 'wb') as file:
          pickle.dump(model.performance_anytime, file)

      file_path = f'Performance/Results/{model_type}/{dataset}/Anytime/pretrain-T{Pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-Mask_{mask_option}_iter{iteration}_selectedMask.pkl'
      with open(file_path, 'wb') as file:
          pickle.dump(model.selected_mask_index, file)

      file_path=f'Performance/Final Models/{model_type}/{base_model}/{dataset}/Anytime/{dataset_name}-{base_model}-pretrain-hidden{hidden_size}-epoch10-Mask_{mask_option}-itter{iteration}.pickle'
      with open(file_path, 'wb') as file:
          pickle.dump(model.all_models_weight, file)

# Weather datasets


## Executable code:


In [ ]:
hidden_size_list = [50]
pretrain_mask_list = ['2']
for hidden in hidden_size_list:
  for pretrain in pretrain_mask_list:
    for iter in range(1,11):
      ###### Config the parameters ######
      base_model = 'GRU'
      batch_size = 1
      hidden_size = hidden
      seq_len = 11
      epoch_size=10
      lr = 0.01
      number_of_tasks=4
      mask_selection_NofBatch = 50 * 128
      input_size=4
      many_to_one = True
      iteration=iter
      mask_init='uniform'
      dataset='Weather'
      dataset_name = "weather_st124_4conf"
      Pretrain_task = '1'
      ###### End of Config the parameters ######
      df = pd.read_csv(os.path.join(f"datasets/{dataset}/", f"{dataset_name}.csv"))
      pretrain_model_addr=f'Performance/Pretrain/{base_model}/{dataset}/After/{dataset}-{base_model}-pretrain-hidden{hidden_size}-epoch10_itter{iteration}.pickle'
      mask_weights=[] #if we have initial masks then reload it here


      mask_option = 'SUM'
      model_type = 'cPB'
      batch_first = True
      low_rank = False
      first_batch = True
      model=cPB(lr = lr ,hidden_size = hidden_size, seq_len = seq_len,base_model = base_model, pretrain_model_addr = pretrain_model_addr,
                mask_weights = mask_weights, mask_init = mask_init, number_of_tasks = number_of_tasks, epoch_size = epoch_size,
                input_size = input_size, mask_option = mask_option, model_type = model_type, batch_first = batch_first, low_rank = low_rank, many_to_one = many_to_one)


      # loop for each task
      x_batch = []
      y_batch = []
      prev_data_x = []
      prev_data_y = []
      for task in range(1, df["task"].max() + 1):
        df_task = df[df["task"] == task]
        df_task = df_task.drop(columns="task")
        # loop based on each batch of data
        batch_cont=0
        x = []
        y = []
        model.weights_copy_anytime(task)
        for i in range(0, len(df_task), batch_size):
          # x = df_task.iloc[0 : i + batch_size, 0:-1].values.astype(np.float32)
          # y = list(df_task.iloc[0 : i + batch_size, -1])
          if len(x_batch)==128:
            if first_batch==True:
              first_batch = False
            x_batch = []
            y_batch = []
            if first_batch == False:
              prev_data_x = x[-seq_len:]
              prev_data_y = y[-seq_len:]
          x_batch.append(df_task.iloc[i, 0:-1].values.astype(np.float32))
          y_batch.append(df_task.iloc[i, -1])
          
          x.append(df_task.iloc[i, 0:-1].values.astype(np.float32))
          y.append(df_task.iloc[i, -1])
          
          # print(len(x))
          if len(x) >= seq_len:
            if i<mask_selection_NofBatch:
              for mask_index in range (0, task):
                model.predict_one(x[-seq_len:],y[-seq_len:],mask_index,task,mask_selection=True)
                if len(x_batch)==128:
                  if first_batch:
                    model.learn_many_anytime(x_batch,y_batch,mask_index)
                  else:
                    new_batch_x = np.concatenate([prev_data_x, x_batch], axis=0) 
                    new_batch_y = np.concatenate([prev_data_y, y_batch], axis=0)           
                    model.learn_many_anytime(new_batch_x,new_batch_y,mask_index)

                  
              batch_cont+=1
            elif i==mask_selection_NofBatch:
              best_mask_index=model.add_new_column_anytime(task)
              model.predict_one(x[-seq_len:],y[-seq_len:],best_mask_index,task,mask_selection=False)
              if len(x_batch)==128:
                  new_batch_x = np.concatenate([prev_data_x, x_batch], axis=0) 
                  new_batch_y = np.concatenate([prev_data_y, y_batch], axis=0)
                  model.learn_many_anytime(new_batch_x,new_batch_y,best_mask_index)
              batch_cont+=1
            elif i>mask_selection_NofBatch:
              batch_cont+=1
              model.predict_one(x[-seq_len:],y[-seq_len:],best_mask_index,task,mask_selection=False)
              if len(x_batch)==128:
                  new_batch_x = np.concatenate([prev_data_x, x_batch], axis=0) 
                  new_batch_y = np.concatenate([prev_data_y, y_batch], axis=0)
                  model.learn_many_anytime(new_batch_x,new_batch_y,best_mask_index)
        all_y = df_task.iloc[:, -1].reset_index(drop=True)
        model.save_final_metrics_anytime(task,all_y)
        model.final_weights_saving()
      model.plotting()


      file_path = f'Performance/Results/{model_type}/{dataset}/Anytime/pretrain-T{Pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-Mask_{mask_option}_iter{iteration}.pkl'
      with open(file_path, 'wb') as file:
          pickle.dump(model.performance_anytime, file)

      file_path = f'Performance/Results/{model_type}/{dataset}/Anytime/pretrain-T{Pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-Mask_{mask_option}_iter{iteration}_selectedMask.pkl'
      with open(file_path, 'wb') as file:
          pickle.dump(model.selected_mask_index, file)

      file_path=f'Performance/Final Models/{model_type}/{base_model}/{dataset}/Anytime/{dataset_name}-{base_model}-pretrain-hidden{hidden_size}-epoch10-Mask_{mask_option}-itter{iteration}.pickle'
      with open(file_path, 'wb') as file:
          pickle.dump(model.all_models_weight, file)